In [ ]:

import os
import math
import logging
from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,
    Trainer,
    TrainingArguments,
    set_seed,
    DataCollatorForLanguageModeling,
    LineByLineTextDataset
)

def train_model():
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)

    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    special_tokens = {
        "bos_token": "<BOS>",
        "eos_token": "<EOS>",
        "pad_token": "<PAD>",
        "additional_special_tokens": ["<superhero>", "<action>", "<drama>", "<thriller>", "<horror>", "<sci_fi>"]
    }
    tokenizer.add_special_tokens(special_tokens)

    model = GPT2LMHeadModel.from_pretrained("gpt2")
    model.resize_token_embeddings(len(tokenizer))

    train_dataset = LineByLineTextDataset(
        tokenizer=tokenizer,
        file_path=r"C:\Users\Dell\Desktop\Movie Plot Generator\6_genre_clean_training_data.txt",
        block_size=128,
    )

    eval_dataset = LineByLineTextDataset(
        tokenizer=tokenizer,
        file_path=r"C:\Users\Dell\Desktop\Movie Plot Generator\6_genre_eval_data.txt",
        block_size=128,
    )

    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    training_args = TrainingArguments(
        output_dir=r"C:\Users\Dell\Desktop\Movie Plot Generator\story_gen_output2",
        overwrite_output_dir=True,
        do_train=True,
        do_eval=True,
        num_train_epochs=3,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        save_steps=500,
        save_total_limit=1,
        logging_steps=100,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    set_seed(42)
    trainer.train()
    trainer.save_model()
    tokenizer.save_pretrained(training_args.output_dir)

train_model()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: viswasudheer01082005 (viswasudheer01082005-visvesvaraya-national-institute-of-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,3.869200
200,3.791500
300,3.687900
400,3.766200
500,3.693400
600,3.646800
700,3.663000
800,3.642800
900,3.595200
1000,3.640700


In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch


def load_model_and_tokenizer(path=r"C:\Users\Dell\Desktop\Movie Plot Generator\story_gen_output2"):
    model = GPT2LMHeadModel.from_pretrained(path, use_safetensors=True)
    tokenizer = GPT2Tokenizer.from_pretrained(path)
    model.eval()
    return model, tokenizer


def generate_story(prompt, model, tokenizer, max_length=300):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    eos_token_id = tokenizer.eos_token_id

    output = model.generate(
        input_ids,
        max_length=max_length,
        do_sample=True,
        top_k=40,
        top_p=0.92,
        temperature=1.0,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=eos_token_id,
        repetition_penalty=1.2,
        no_repeat_ngram_size=2
    )

    story = tokenizer.decode(output[0], skip_special_tokens=True)
    return story

model, tokenizer = load_model_and_tokenizer()
story = generate_story("<BOS> <action>", model, tokenizer)
print(story)

C:\Users\Dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


  In the year AD 6, a large white dog is brought to King Philip by an animal trader. The two set out on their journey but are interrupted by mysterious flying creatures which come across them as they were meant for hunting or catching prey. To stop them and save mankind from extinction one of Drayson's most dangerous creatures returns with his henchmen. 


In [ ]:
pip install gradio

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import gradio as gr

# Load model and tokenizer
model_path = "/content/drive/MyDrive/ML PROJ/story_gen_output2"
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model.eval()


genre_backgrounds = {
    "horror": "/content/drive/MyDrive/ML PROJ/pics/horr.jpg",
    "action": "https://i.ibb.co/vs5Q2vp/action.jpg",
    "sci_fi": "https://i.ibb.co/n1Y8BQX/scifi.jpg",
    "drama": "https://i.ibb.co/X3XmTc4/drama.jpg",
    "superhero": "https://i.ibb.co/98R3jLt/superhero.jpg",
    "thriller": "https://i.ibb.co/t4pZMPf/thriller.jpg"
}


def generate_story(genre, max_tokens=100, temperature=0.9):
    genre_token = f"<{genre.lower()}>"
    input_ids = tokenizer.encode(genre_token, return_tensors="pt")

    output = model.generate(
        input_ids,
        max_new_tokens=max_tokens,
        temperature=temperature,
        top_k=50,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id
        # eos_token_id=tokenizer.eos_token_id
    )

    story = tokenizer.decode(output[0], skip_special_tokens=True)

    background_url = genre_backgrounds.get(genre, "")

    styled_output = f"""
    <div style='padding: 20px;
                color: white;
                font-family: "Georgia", serif;
                background: linear-gradient(to right, rgba(0,0,0,0.7), rgba(0,0,0,0.9)),
                            url({background_url});
                background-size: cover;
                background-repeat: no-repeat;
                border-radius: 15px;
                box-shadow: 0 0 15px rgba(0,0,0,0.6);'>
        <h2>{genre.title()} Story</h2>
        <p style='font-size: 18px; line-height: 1.6;'>{story}</p>
    </div>
    """
    return styled_output

# Genre list
genre_list = ["horror", "action", "sci_fi", "drama", "superhero", "thriller"]

# Gradio interface
with gr.Blocks(theme=gr.themes.Soft(primary_hue="rose")) as demo:
    gr.Markdown("""
        <h1 style='text-align: center; font-family: Arial; color: #880E4F;'>🎬 Movie Story Generator</h1>
        <p style='text-align: center;'>Choose a genre and let the Model write a story for you!</p>
    """)

    with gr.Row():
        genre = gr.Dropdown(choices=genre_list, label="Select Genre")
        max_tokens = gr.Slider(minimum=50, maximum=500, step=10, value=100, label="Max Tokens")
        temperature = gr.Slider(minimum=0.5, maximum=1.5, step=0.1, value=0.9, label="Creativity (Temperature)")

    output = gr.HTML(label="Generated Story")

    generate_btn = gr.Button("Generate Story ✨")

    generate_btn.click(fn=generate_story,
                       inputs=[genre, max_tokens, temperature],
                       outputs=output)

demo.launch()


ModuleNotFoundError: No module named 'gradio'